In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to thought signatures

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fthinking%2Fintro_thought_signatures.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/thinking/intro_thought_signatures.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Authors |
| --- |
| [Eric Dong](https://github.com/gericdong) |

## Overview

A thought signature is an encrypted representation of the model's internal reasoning process for a given turn in a conversation. By passing this signature back to the model in subsequent requests, you provide it with the context of its previous thoughts, allowing it to build upon its reasoning and maintain a coherent line of inquiry.

Thought signatures provide a powerful mechanism to maintain context in multi-turn conversations, enabling the model to tackle more complex, multi-step tasks that require reasoning and the use of external tools through function calling.


### Objectives

This tutorial explores the thought signatures feature of the Gemini API.


## Getting Started

### Install the Google Gen AI SDK for Python


In [ ]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Authenticate to Vertex AI on Google Cloud

You'll need to set up authentication by choosing **one** of the following methods:

1.  **Use a Google Cloud Project:** (Recommended for most users)
    - See instructions [Set up a project and development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)
2.  **Use a Vertex AI API Key (Express Mode):** For quick experimentation.
    - [Get an API Key](https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview)
    - See tutorial [Getting started with Gemini using Vertex AI in Express Mode](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_express.ipynb).

This tutorial uses a Google Cloud Project for authentication.

In [ ]:
import os

# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "global"

### Import libraries


In [ ]:
from IPython.display import Markdown, display
from google import genai
from google.genai.types import (
    Content,
    GenerateContentConfig,
    Part,
    ThinkingConfig,
    Tool,
)

### Connect to the generative AI service on Vertex AI

In [ ]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Supported Models

Learn more about [supported Gemini models](https://cloud.google.com/vertex-ai/generative-ai/docs/models). This tutorial uses the Gemini 2.5 Flash model.

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

## Use Thought Signatures

When thinking is enabled, the API response includes a `thought_signature` field containing an encrypted representation of the model's reasoning. When a function's execution result is sent back to the server, including the `thought_signature` allows the model to restore its previous thinking context, which will likely improve function calling performance.

Example:

```
thought_signature=b"\n\xa9\x06\x01\xcb...'
```


### Use Cases
The primary and most powerful use case for thought signatures is to enable sophisticated, multi-turn interactions with external tools. Imagine a scenario where a user's request requires multiple steps to resolve, such as:

- Finding the current weather.
- Based on the weather, deciding on an appropriate activity.
- Booking a reservation for that activity.

Each of these steps might involve calling a different function. Without thought signatures, the model would treat each step as an isolated request, losing the crucial context of the user's initial intent and the results of previous function calls. With thought signatures, the model can chain these calls together, understanding the dependencies and making informed decisions at each step.


## Example: Conditional Thermostat Control

In this scenario, a user wants to set a thermostat based on the current weather. The request is: "If it's too hot or too cold in London, set the thermostat to a comfortable temperature."

This requires the model to:

- Call a tool to get the weather in London.
- Use the returned weather information to decide if another tool needs to be called.
-Call the tool to set the thermostat if the condition is met.

### **Step 1**: Define Functions and Tools

First, we define the two functions the model can use:  `get_current_temperature` and `set_thermostat_temperature`.

In [ ]:
# Mock Function Implementations


def get_current_temperature(location: str) -> dict:
    """Gets the current weather temperature for a given location."""
    print(f"Tool Call: get_current_temperature(location={location})")
    # Mocking a real API call.
    return {"temperature": 30, "unit": "celsius"}


def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature."""
    print(f"Tool Call: set_thermostat_temperature(temperature={temperature})")
    # In a real app, this would interact with a thermostat API.
    return {"status": "success"}


# Function Declarations for the model


get_weather_declaration = {
    "name": "get_current_temperature",
    "description": "Gets the current weather temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {"location": {"type": "string"}},
        "required": ["location"],
    },
}

set_thermostat_declaration = {
    "name": "set_thermostat_temperature",
    "description": "Sets the thermostat to a desired temperature.",
    "parameters": {
        "type": "object",
        "properties": {"temperature": {"type": "integer"}},
        "required": ["temperature"],
    },
}

# Create a single Tool with both functions
# The model will choose which, if any, to call.
thermostat_tools = Tool(
    function_declarations=[
        get_weather_declaration,
        set_thermostat_declaration,
    ]
)

### **Step 2**: First Turn - Get the Weather

We send the initial prompt to the model. We expect it to call the `get_current_temperature` function and return a thought signature to maintain the context of the user's conditional request.

In [ ]:
# The user's initial multistep request
prompt = """
If it's too hot or too cold in London, set the temperature to a comfortable level.
Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification.
"""

contents = [Content(role="user", parts=[Part(text=prompt)])]

# Send the first request
config = GenerateContentConfig(
    tools=[thermostat_tools],
    thinking_config=ThinkingConfig(
        include_thoughts=True,  # Optional, include_thoughts can only be enabled when thinking is enabled
    ),
)

response_turn_1 = client.models.generate_content(
    model=MODEL_ID, config=config, contents=contents
)

In [ ]:
def process_tool_call(response):
    """Processes a model's response to extract and execute a tool call.

    Args:
        response: The response object from the model.

    Returns:
        A tuple containing the tool_call object and the result of its execution.
        Returns (None, None) if no function call is found.
    """
    tool_call = None
    result = None

    # Extract the function call, thought signatures, thought summary
    # from the response parts
    for part in response.candidates[0].content.parts:
        if part.function_call:
            tool_call = part.function_call
        if hasattr(part, "thought_signature") and part.thought_signature:
            display(Markdown(f"**Thought Signature**: {part.thought_signature}"))
        if part.thought:
            display(Markdown(f"**Thought Summary**: {part.text}"))
        else:
            display(Markdown(f"**Answer**: {part.text}"))

    if tool_call:
        if tool_call.name == "get_current_temperature":
            result = get_current_temperature(**tool_call.args)
            display(Markdown(f"**Tool Response**: {result}"))
        elif tool_call.name == "set_thermostat_temperature":
            result = set_thermostat_temperature(**tool_call.args)
            display(Markdown(f"**Tool Response**: {result}"))
    else:
        print("No valid tool call found in the response.")

    return tool_call, result

In [ ]:
# Process the response
print("--- Turn 1 ---")
tool_call_1, result_1 = process_tool_call(response_turn_1)

### **Step 3**: Second Turn - Set the Thermostat

Now, we send the model's previous response (containing the thought signature) and the result from our first tool call back to the model. Using this context, the model will decide if it needs to call the `set_thermostat_temperature` function.

There is no additional work for you to do. The `"thought_signature"` part will be sent back in the model turn's `Content`, it is also handled transparently by
automatic function calling.

In [ ]:
# Append the model's function call message from turn 1 (this includes the signature)
contents.append(response_turn_1.candidates[0].content)

# Append the result of the function execution from turn 1
contents.append(
    Content(
        role="tool",
        parts=[
            Part.from_function_response(
                name=tool_call_1.name,
                response=result_1,
            )
        ],
    )
)

# Send the second request
response_turn_2 = client.models.generate_content(
    model=MODEL_ID, config=config, contents=contents
)

# Process the response
tool_call_2, result_2 = process_tool_call(response_turn_2)

### **Step 4**: Generate a user-friendly response

Generate a user-friendly response, informed by the action it just took.

In [ ]:
# Append this turn's messages to history for a final response
contents.append(response_turn_2.candidates[0].content)
contents.append(
    Content(
        role="tool",
        parts=[
            Part.from_function_response(
                name=tool_call_2.name,
                response=result_2,
            )
        ],
    )
)

# Get the final text response
final_response = client.models.generate_content(
    model=MODEL_ID, config=config, contents=contents
)

display(Markdown(f"**Final Response**: {final_response.text}"))

This multi-turn process, held together by passing the model's response history (including the encrypted thought signature) back each time, allows the model to perform complex, sequential tasks.